In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold
import time

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

In [4]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [5]:
from preprocess import preprocess

In [6]:
X_train, X_test, y_train = preprocess(df_train, df_test, use_ohe=True, filter_features=True, use_scaling=True)

In [7]:
bnb_pipe = Pipeline([
    ('bnb', BernoulliNB())
])

dt_pipe = Pipeline([
    ('dt', DecisionTreeClassifier())
])

etc_pipe = Pipeline([
    ('etc', ExtraTreesClassifier())
])

gnb_pipe = Pipeline([
    ('gnb', GaussianNB())
])

kn_pipe = Pipeline([
    ('kn', KNeighborsClassifier())
])

svc_pipe = Pipeline([
    ('svc', LinearSVC())
])

lr_pipe = Pipeline([
    ('lr', LogisticRegression())
])

rf_pipe = Pipeline([
    ('rf', RandomForestClassifier())
])

In [8]:
pipelines = [bnb_pipe, dt_pipe, etc_pipe, gnb_pipe, kn_pipe, lr_pipe, rf_pipe]

In [11]:
def test_pipeline(X, y, pipeline):
    rskf = StratifiedKFold(n_splits=5, random_state=1)
    rmse_scores = []
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pipeline.fit(X_train, y_train)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, pipeline.predict(X_test))))

    print("kfolds rmse: {0}, mean rmse: {1}".format(
        str([str(round(x, 3)) for x in sorted(rmse_scores)]),
        round(np.mean(rmse_scores), 3)
    ))

In [10]:
objects_to_calculate = 50000
for pipeline in pipelines:
    print("Model:", type(pipeline[0]).__name__)
    test_pipeline(X_train[:objects_to_calculate], y_train[:objects_to_calculate], pipeline)
    print('-----------------------------------------------------------------------')

Model: BernoulliNB
kfolds rmse: ['1.572', '1.579', '1.581', '1.586', '1.589'], mean rmse: 1.581
-----------------------------------------------------------------------
Model: DecisionTreeClassifier
kfolds rmse: ['1.275', '1.293', '1.295', '1.303', '1.321'], mean rmse: 1.297
-----------------------------------------------------------------------
Model: ExtraTreesClassifier


/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


kfolds rmse: ['1.266', '1.268', '1.27', '1.271', '1.275'], mean rmse: 1.27
-----------------------------------------------------------------------
Model: GaussianNB
kfolds rmse: ['1.33', '1.36', '1.47', '1.627', '1.646'], mean rmse: 1.487
-----------------------------------------------------------------------
Model: KNeighborsClassifier
kfolds rmse: ['1.347', '1.352', '1.355', '1.362', '1.373'], mean rmse: 1.358
-----------------------------------------------------------------------
Model: LogisticRegression


/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/whiteRa2bit/anaconda3/lib/python3.7/site-package

kfolds rmse: ['1.573', '1.592', '1.597', '1.6', '1.603'], mean rmse: 1.593
-----------------------------------------------------------------------
Model: RandomForestClassifier


/home/whiteRa2bit/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


kfolds rmse: ['1.222', '1.228', '1.228', '1.238', '1.241'], mean rmse: 1.231
-----------------------------------------------------------------------


In [23]:
classifiers = [
    ("bnb", bnb_pipe),
    ("dt", dt_pipe),
    ("etc", etc_pipe),
    #("gnb", gnb_pipe),
    ("kn", kn_pipe),
    #("svc", svc_pipe),
    #("lr", lr_pipe),
    ("rf", rf_pipe),
]

mixed_pipe = Pipeline([
    ("voting", VotingClassifier(classifiers, voting="soft"))
])

In [24]:
test_pipeline(X_train_transformed.values[:objects_to_calculate], y_train[:objects_to_calculate], mixed_pipe)

kfolds rmse: ['1.208', '1.214', '1.215', '1.227', '1.228'], mean rmse: 1.218


In [2]:
from scipy.stats import rankdata
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from itertools import cycle
from sklearn.base import BaseEstimator, ClassifierMixin

class ShuffleVoter(BaseEstimator, ClassifierMixin):  
    """
       scikit-learn based voting aggregation ensembling.
       Using bootstrapping creates a set of models, differing only by which data sample they are fed
       """

    def __init__(self, models):
        """
        model - base model ( or a pipeline ) ( unfitted )
        """
        self.models = models
        
    def ensemble_predictions(self, predictions, weights, type_="harmonic"):
        """
        Combines probabilistic class estimates using a variety of strategies.
        Linear, harmonic, geometric and rank averaging are supported at this moment. 
        Insipred by well known Abhishek's kernel on Kaggle 
        model - base model ( or a pipeline ) ( unfitted )
        """
        assert np.isclose(np.sum(weights), 1.0)
        if type_ == "linear":
            res = np.average(predictions, weights=weights, axis=0)
        elif type_ == "harmonic":
            res = np.average([1 / p for p in predictions], weights=weights, axis=0)
            return 1 / res
        elif type_ == "geometric":
            numerator = np.average(
                [np.log(p) for p in predictions], weights=weights, axis=0
            )
            res = np.exp(numerator / sum(weights))
            return res
        elif type_ == "rank":
            res = np.average([rankdata(p) for p in predictions], weights=weights, axis=0)
            return res / (len(res) + 1)
        return res


    def fit( self, X, y, n_boots = 14, test_size = 100 ):
        """
        n_boots - number of bootstrapping iterations ( and respective models built)
        """
        self.clfs  = []
        for i, model in zip(range(n_boots), cycle(self.models)):
            X_tr, X_te, y_tr, y_te = train_test_split( X, y, test_size=test_size, random_state=3521 + i*10)

            pa_clf = model
            pa_clf.fit(X_tr, y_tr)

            self.clfs.append(pa_clf)

    def predict( self, X, ensemble_type = 'rank'):
        # TODO: nonuniform weights
        
        n_boots = len( self.clfs)
        preds = [ clf.predict(X) for clf in self.clfs ]
        return self.ensemble_predictions( preds, np.ones(n_boots)*(1/float(n_boots)), ensemble_type)
    
    def predict_proba( self, X, ensemble_type = 'rank' ):
        n_boots = len( self.clfs)
        preds = [ clf.predict_proba(X) for clf in self.clfs ]
        return self.ensemble_predictions( preds, np.ones(n_boots)*(1/float(n_boots)), ensemble_type)
